# Sentinel 3 - OLCI Data Download

This notebook presents the tools for the data access and download of Sentinel-3 A/B data OLCI images using the WEkEO API.

## 1. Define Workspace

In [ ]:
import base64

import os
import zipfile
import shutil
from dotenv import load_dotenv
load_dotenv()

# API requests config
import requests

# Widgets and maps view
# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as wg
# from IPython.display import display

In [ ]:
cwd = {
    'S3_images': './in/satellite_imagery/S3',
    'EUMETSAT_images': './in/satellite_imagery/EUMETSAT',
    'simile_lakes': './vector/simile_laghi'
}   

## 2. Setup HDA Client

For setting up the credentials for the HDA API, you must edit the `./notebooks/.env` file with you WEkEO user. It is possible to create a user account in the [WEkEO](https://www.wekeo.eu/) platform by going to the **Register** option and complitening the form to create an account

**Warning:** The download of the satellite images using the API is limited on the availability of the server. In case there is no response from the server, it is possible retrieving the images from the [Copernicus Access Hub](https://scihub.copernicus.eu/dhus/#/home).

In [ ]:
# Import the credentials from the configuration file in the notebooks directory
url = os.environ['HDA_URL']
user = os.environ['HDA_USER']
password = os.environ['HDA_PASSWORD']

In [ ]:
credentials = user+": "+password
credentials_bytes = credentials.encode('ascii')
base64_bytes = base64.b64encode(credentials_bytes)
base64_credentials = base64_bytes.decode('ascii')

Create a string of a call to **GET** /gettoken to get a token. Execute it with os.popen in order to store output as variable

In [ ]:
header = {'authorization' : 'Basic '+base64_credentials}
response = requests.get(url+'/gettoken', headers=header)
response = response.json()

Before data can be accessed, the Copernicus Terms and Conditions must be accepted. This needs to be done only once.

In [ ]:
header = {
    'accept': 'application/json',
    'authorization' : response['access_token']
}
requests.put(url+'/termsaccepted/Copernicus_General_License', headers=header)

In [ ]:
parameters = 'url: '+url+'\nuser: '+user+'\npassword: '+password+'\ntoken: '+response['access_token']

In [ ]:
# Creation of the client .hdarc config file in the $HOME direcotry used by the hda API library. It is sufficient to run this step once.
with open(os.path.join(os.environ['HOME'],'.hdarc'), 'w') as fp:
    fp.write(parameters)

## 3. Request Sentinel-3 A/B images

In this section, we will review the parameters for the composition of a new request of the satellite images using the [hda API](https://www.wekeo.eu/docs/harmonised-data-access-api).

In [ ]:
from hda import Client

In [ ]:
#Avoid printing secrets
c = Client()

In [ ]:
# Import necessary libraries for the project

# Data
import pandas as pd
import geopandas as gpd

# Plotting
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

Import the reference layer for determining the bound of the query. In this case, it corresponds to the lakes extent.

In [ ]:
gdf = gpd.read_file(os.path.join(cwd['simile_lakes'],'simile_laghi.shp'))

In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))
# Plot lakes vector dataset
gdf.plot(ax=ax)
ax.grid()
# Create scale bar
scalebar = ScaleBar(0.001, "km", length_fraction=0.25)
ax.add_artist(scalebar)

In [ ]:
# It is necessary to have the query coordinates in web mercator
gdf = gdf.to_crs("EPSG:4326")
# Extract the information from the bounding box of the layer
x_min = min(gdf.bounds['minx'])
x_max = min(gdf.bounds['maxx'])
y_min = min(gdf.bounds['miny'])
y_max = min(gdf.bounds['maxy'])

Define the dictionary with the parameters for the file to import. In this case, the image download refer to the *"OLCI Level 1B Full Resolution - Sentinel-3"* products available through the API. To find more information about the avaliable datasets in the platform you are welcome to visit the [WEkEO data viewer](https://www.wekeo.eu/data?view=viewer). Make sure to specify the dates of interest for the processing.

In [ ]:
query = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:OL_1_EFR___",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        x_min,
        y_min,
        x_max,
        y_max
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2022-12-16T00:00:00.000Z",
      "end": "2022-12-31T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "platformname",
      "value": "Sentinel-3A"
    },
    {
      "name": "producttype",
      "value": "OL_1_EFR___"
    },
    {
      "name": "timeliness",
      "value": "NT"
    }
  ]
}

In [ ]:
# The following line runs the query
matches = c.search(query)

# The following line prints the products returned by the query
print(matches)

In [ ]:
#The download starts. All the products found in the query are downloaded consecutively
matches.download()

In [ ]:
for product in matches.__dict__['results']:
    shutil.move(product['filename'], cwd['S3_images'])

**Warning:** Make sure that the folder contains only the compressd filed with the satellite imagery to avoid running into an error.

In [ ]:
def extractZipFile(image_dir, file_path):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(image_dir)
        os.remove(file_path)

In [ ]:
for root, dirs, files in os.walk(cwd['S3_images']):
    for zip_name in files:
        file_path = os.path.join(root,zip_name)
        # Exception required for products retrieved through the HDA API. The products download include the .SEN3 extension and not a compressed file format
        if zip_name.endswith('.SEN3'):
            print(f'Currently extracting: {zip_name}')
            os.rename(os.path.join(cwd['S3_images'],zip_name),os.path.join(cwd['S3_images'],zip_name.split('.')[0]+'.zip'))
            file_path = os.path.join(cwd['S3_images'],zip_name.split('.')[0]+'.zip')
            extractZipFile(cwd['S3_images'], file_path)
        elif zip_name.endswith('.zip'):
            print(f'Currently extracting: {zip_name}')
            extractZipFile(cwd['S3_images'], file_path)

# Download EUMETSAT

In [ ]:
query = {
  "datasetId": "EO:EUM:DAT:SENTINEL-3:OL_2_WFR___",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        x_min,
        y_min,
        x_max,
        y_max
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2022-12-13T00:00:00.000Z",
      "end": "2022-12-31T00:00:00.000Z"
    }
  ],
  "stringChoiceValues": [
    {
      "name": "sat",
      "value": "Sentinel-3B"
    },
    {
      "name": "timeliness",
      "value": "NT"
    },
    {
      "name": "type",
      "value": "OL_2_WFR___"
    }
  ]
}

In [ ]:
# The following line runs the query
matches = c.search(query)

# The following line prints the products returned by the query
print(matches)

In [ ]:
#The download starts. All the products found in the query are downloaded consecutively
matches.download()

In [ ]:
for product in matches.__dict__['results']:
    shutil.move(product['filename'], cwd['EUMETSAT_images'])

In [ ]:
for root, dirs, files in os.walk(cwd['EUMETSAT_images']):
    for zip_name in files:
        file_path = os.path.join(root,zip_name)
        print(f'Currently extracting: {zip_name}')
        # Exception required for products retrieved through the HDA API. The products download include the .SEN3 extension and not a compressed file format
        if zip_name.endswith('.SEN3'):
            os.rename(os.path.join(cwd['EUMETSAT_images'],zip_name),os.path.join(cwd['EUMETSAT_images'],zip_name.split('.')[0]+'.zip'))
            file_path = os.path.join(cwd['EUMETSAT_images'],zip_name.split('.')[0]+'.zip')
            extractZipFile(cwd['EUMETSAT_images'], file_path)
        elif zip_name.endswith('.zip'):
            extractZipFile(cwd['EUMETSAT_images'], file_path)